In [ ]:
%matplotlib ipympl
import numpy as np
from numpy import pi, sqrt
import xml.etree.ElementTree as ET
import ase.io
from pymatgen.io.vasp.outputs import Vasprun
import matplotlib.pyplot as plt
import pandas as pd
np.set_printoptions(suppress=True, threshold=np.inf)
pd.options.display.max_rows = None

# Helper Functions

In [ ]:
def get_Born(vasprun_born='vasprun.xml', natom=45):
    """
    Get Born effective charges from vasprun.xml.
    """

    tree = ET.parse(vasprun_born)
    root = tree.getroot()

    born = np.zeros((natom,3,3))

    x = root.findall("./calculation/array[@name='born_charges']/set/v")
    for i,j in enumerate(x):
        born[i//3][i%3] = j.text.split()

    asr = np.sum(born, axis=0) / natom      # violation of acoustic sum rule.
    born = np.array([i-asr for i in born])

    return born

In [ ]:
def get_phonons(qpoint_file='qpoints.yaml', poscar='POSCAR'):
    """
    Get phonon frequencies and eigen-displacements from finite differences in Phonopy or DFPT in VASP.    
    """
    
    # Using Phonopy:
    # with open(qpoint_file, 'r') as file:
    #     data = yaml.safe_load(file)

    # freqs = []
    # eigs = []
    
    # for band in data['phonon'][0]['band']:
    #     freqs.append(band['frequency'])     # frequency of band
    #     eig = np.array(band['eigenvector'])      # eigenvector of band
    #     eig = eig[:,:,0]

    #     structure = ase.io.read(poscar)
    #     mass = structure.get_masses()
    #     eig = eig / np.sqrt(np.tile(mass, (3,1)).T)     # eigendisplacements = eigenvectors / sqrt(mass).
    #     eigs.append(eig)

    # Using DFPT:
    run = Vasprun(qpoint_file)
    freqs = np.flip(np.sqrt(-run.normalmode_eigenvals))
    eigs = np.flip(run.normalmode_eigenvecs, axis=0)

    structure = ase.io.read(poscar)
    mass = structure.get_masses()
    eigs = eigs / np.sqrt(np.tile(mass, (3,1)).T)
    

    return (freqs, eigs)

In [ ]:
def calculate_polarity(born, phonon):
    """
    Calculate phonon mode polarity from mode eigen-displacements and Born effective charges.
    """
    return np.einsum('ijk,ik->j', born, phonon)

In [ ]:
def susceptibility_derivatives(disp=0.01, natoms=45):
    """
    Calculate d chi / d tau using finite differences.
    """
    d_chi_d_tau = []

    for i in range(natoms*3):
        out_m = Vasprun(
            f"{i}/minus/vasprun.xml",
            parse_dos=False,
            parse_eigen=False,
            parse_projected_eigen=False,
            parse_potcar_file=False)
        chi_m = (np.array(out_m.epsilon_static) - 1) / (4*np.pi)

        out_p = Vasprun(
            f"{i}/plus/vasprun.xml",
            parse_dos=False,
            parse_eigen=False,
            parse_projected_eigen=False,
            parse_potcar_file=False)
        chi_p = (np.array(out_p.epsilon_static) - 1) / (4*np.pi)

        d_chi_d_tau.append((chi_p - chi_m)/(2*disp))

    d_chi_d_tau = np.array(d_chi_d_tau)
    d_chi_d_tau = np.reshape(d_chi_d_tau, (natoms, 3, 3, 3))
    asr = np.sum(d_chi_d_tau, axis=0) / natoms      # violation of acoustic sum rule.
    for i in range(natoms):
        d_chi_d_tau[i] = d_chi_d_tau[i] - asr

    return d_chi_d_tau

In [ ]:
def calculate_Raman(d_chi_d_tau, phonon):
    """
    Calculate alpha from d_chi_d_tau and phonon eigen-displacements.
    Note: This number needs to be multiplied by sqrt(volume) to get the real Raman susceptibility.
    """
    return np.einsum('ijkl,ij->kl', d_chi_d_tau, phonon)

In [ ]:
def to_voigt(r):
    """
    Convert a 3x3x3 tensor to Voigt notation.
    """
    r_voigt = np.zeros((6,3))
    r_voigt[0,:] = r[0,0,:]
    r_voigt[1,:] = r[1,1,:]
    r_voigt[2,:] = r[2,2,:]
    r_voigt[3,:] = (r[1,2,:] + r[2,1,:]) / 2
    r_voigt[4,:] = (r[0,2,:] + r[2,0,:]) / 2
    r_voigt[5,:] = (r[0,1,:] + r[1,0,:]) / 2
    return r_voigt

In [ ]:
def average_configs(r):
    """
    Average over 4 orientations to enforce C4 symmetry.
    Only applicable for SBN.
    """
    def rotate_90(r):
        rot = [[0,-1,0],[1,0,0],[0,0,1]]
        return np.einsum('ai,bj,ck,ijk->abc', rot, rot, rot, r)
    
    return (r + rotate_90(r) + rotate_90(rotate_90(r)) + rotate_90(rotate_90(rotate_90(r)))) / 4

# Ionic Pockels Tensor

In [ ]:
# Get tensors:
natom = 45

born = get_Born("../../03-Dielectric/LDA_opt/vasprun.xml", natom=natom)
# freqs, eigs = get_phonons(qpoint_file="../../02-Phonons/1x1x3/qpoints.yaml")    # Finite difference phonons.
freqs, eigs = get_phonons(qpoint_file="../../02.2-Phonons_DFPT/LDA/vasprun.xml")  # DFPT phonons.
d_chi_d_tau = susceptibility_derivatives(0.01, 45)

In [ ]:
# Calculate Pockels tensor and Raman susceptibility tensor:
structure = ase.io.read("POSCAR")
V = structure.get_volume()

run = Vasprun(
            "../../03-Dielectric/LDA_opt/vasprun.xml", parse_dos=False,
            parse_eigen=False,
            parse_projected_eigen=False,
            parse_potcar_file=False)
n = 1/np.linalg.eigvals(run.epsilon_static)
n = np.outer(n, n)

r = np.zeros((natom*3-3,3,3,3))
raman_tensors = np.zeros((natom*3-3,3,3))

for i in range(natom*3-3):
    polarity = calculate_polarity(born, eigs[i+3])
    raman_tensor = calculate_Raman(d_chi_d_tau, eigs[i+3])
    raman_tensors[i,:,:] = raman_tensor*sqrt(V)

    r[i,:,:,:] = np.einsum('ij,k->ijk', raman_tensor*n, polarity)
    r[i,:,:,:] = r[i,:,:,:] / freqs[i+3]**2


# Unit conversion:
# Conver r to atomic units used by Abinit.
# 1 amu = 1822.89 m_e; 1 Angstrom = 1.88973 Bohr; 1 THz = 1.51982985E-4 Ha.
r = r / (1822.89 * 1.88973 * 1.51982985E-4**2)
# Convert r from atomic units to pm/V using formula found in Abinit source code. 
eps0 = 1 / (4*pi*0.0000001*299792458.0**2)
e_Cb = 1.602176487e-19
Bohr_Ang = 0.52917720859
fac = -16*pi*pi*eps0*(Bohr_Ang**2)*1.0E-8/(e_Cb)
r = r * fac

r = [average_configs(r[i]) for i in range(len(r))]  # Average over 4 orientations to enforce C4 symmetry.
r = [to_voigt(r[i]) for i in range(len(r))]
r_total = np.sum(r[1:], axis=0)
np.savetxt('r_total.csv', r_total, delimiter=',')
r_total

In [ ]:
# Plot r_33 and alpha_33:
plt.close()
fig, ax1 = plt.subplots()
ax1.plot([np.abs(raman_tensors[i][2,2]) for i in range(len(raman_tensors))][1:], color='blue')
ax2 = ax1.twinx()
ax2.plot([r[i][2,2] for i in range(len(r))][1:], color='orange')
ax1.set_xlabel('Mode number')
ax1.set_ylabel('alpha', color='blue')
ax2.set_ylabel('r', color='orange')